In [41]:
%load_ext autoreload
%autoreload 2

In [52]:
from src.processors.Preprocessor import Preprocessor
from src.model_experiemtation.ModelExperimentor import TFModelExperimentor

import datetime
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import seaborn.objects as so
from sklearn.model_selection import train_test_split
import tensorflow as tf
from time import sleep

#validate GPU detection
print("TensorFlow version:", tf.__version__)
for gpu in tf.config.list_physical_devices('GPU'):
    print(gpu)


TensorFlow version: 2.12.0
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


In [40]:
PATH = '/home/mds8301/gaby_test/processors/unit_test_processor.pkl'
proc = Preprocessor().load_processor(PATH)
proc.one_hot_encode(labels = ['event', 'sensor'])


In [42]:
data_under_20 = proc.processed_data.query('trial<=20')
data_over_20 = proc.processed_data.query('trial>20')


In [43]:
X_train, y_train = data_under_20.drop(columns = ['signal']), data_under_20.signal
X_test, y_test = data_over_20.drop(columns = ['signal']), data_over_20.signal

In [49]:
X_train

mouse_id  day       time  trial  event_avoid  event_cue  event_escape  \
0       310_909    7 -25.000000      0            0          0             0   
1       310_909    7 -24.999018      0            0          0             0   
2       310_909    7 -24.998034      0            0          0             0   
3       310_909    7 -24.997051      0            0          0             0   
4       310_909    7 -24.996067      0            0          0             0   
...         ...  ...        ...    ...          ...        ...           ...   
228880  310_907    7  19.996067      4            0          0             0   
228881  310_907    7  19.997051      4            0          0             0   
228882  310_907    7  19.998034      4            0          0             0   
228883  310_907    7  19.999018      4            0          0             0   
228884  310_907    7  20.000000      4            0          0             0   

        event_shock  sensor_D1  sensor_D2  sensor_DA  
0                 1          0          1          0  
1                 1          0          1          0  
2                 1          0          1          0  
3                 1          0          1          0  
4                 1          0          1          0  
...             ...        ...        ...        ...  
228880            1          0          0          1  
228881            1          0          0          1  
228882            1          0          0          1  
228883            1          0          0          1  
228884            1          0          0          1  

[66101988 rows x 11 columns]

In [51]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LayerNormalization, LSTM
from tensorflow.keras import Sequential

from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError

sequential_model = Sequential(
    [
        Dense(128, activation='relu', name="dense_1"),
        Dense(1, activation='relu', name="dense_2")
    ]
)


2023-05-24 15:56:39.082432: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38222 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:06:00.0, compute capability: 8.0
2023-05-24 15:56:39.084171: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38222 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:2f:00.0, compute capability: 8.0


In [56]:
logs_dir = "/projects/p31961/dopamine_modeling/results/logs/training_logs/" + \
        datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logs_dir, histogram_freq=1)

In [57]:
sequential_model.compile(
    optimizer="adam", loss="mean_squared_error", metrics=["mse, accuracy"])
sequential_model.fit(X_train, y_train, epochs=200,
                 batch_size=30, callbacks=[tensorboard_callback])
prediction = sequential_model.predict(X_test)
sequential_model.evaluate(X_test, y_test)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).